Contributors: Matthew Zimolzak, Li Zhou
<br /><br />In this notebook we are trying to scrape some basic statistics from the website https://www.basketball-reference.com/.  The method we will be using is presented by Oscar Sanchez in one of his articles on Towards Data Science (https://towardsdatascience.com/web-scraping-nba-stats-4b4f8c525994).

Afterwards, we will be retrieving data from https://sportsdata.io.  The JSON formatted data returned from the API can be found at the following links. <br />
    Player data: https://api.sportsdata.io/v3/nba/scores/json/Players?key=a3824595f2f740dbb21dd847e49ba332 <br />
    Team data: https://api.sportsdata.io/v3/nba/scores/json/teams?key=a3824595f2f740dbb21dd847e49ba332 <br />
    Stadium data: https://api.sportsdata.io/v3/nba/scores/json/Stadiums?key=a3824595f2f740dbb21dd847e49ba332 <br /><br />
**Note:** If you are going to run this notebook, you will likely need to change the filepaths when writing to a CSV (or comment them out if you don't need a CSV).  You can find them in cells 12 and 17.

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import requests
import json
from pandas.io.json import json_normalize

# HTML Webscraping

In [2]:
#We're first going to demonstrate this technique to make sure that we're achieving the intended output

# NBA season we will be analyzing
year = 2019

# URL page we will scraping
url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html".format(year)

# this is the HTML from the given URL
html = urlopen(url)
soup = BeautifulSoup(html)

In [3]:
# use findALL() to get the column headers
soup.findAll('tr', limit=2)

# use getText()to extract the text we need into a list
headers = [th.getText() for th in soup.findAll('tr', limit=2)[0].findAll('th')]

# exclude the first column as we will not need the ranking order from Basketball Reference for the analysis
headers = headers[1:]
headers

['Player',
 'Pos',
 'Age',
 'Tm',
 'G',
 'GS',
 'MP',
 'FG',
 'FGA',
 'FG%',
 '3P',
 '3PA',
 '3P%',
 '2P',
 '2PA',
 '2P%',
 'eFG%',
 'FT',
 'FTA',
 'FT%',
 'ORB',
 'DRB',
 'TRB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PTS']

In [4]:
# avoid the first header row
rows = soup.findAll('tr')[1:]
player_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

In [5]:
stats = pd.DataFrame(player_stats, columns = headers)
stats.head(10)

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Álex Abrines,SG,25,OKC,31,2,19.0,1.8,5.1,.357,...,.923,0.2,1.4,1.5,0.6,0.5,0.2,0.5,1.7,5.3
1,Quincy Acy,PF,28,PHO,10,0,12.3,0.4,1.8,.222,...,.700,0.3,2.2,2.5,0.8,0.1,0.4,0.4,2.4,1.7
2,Jaylen Adams,PG,22,ATL,34,1,12.6,1.1,3.2,.345,...,.778,0.3,1.4,1.8,1.9,0.4,0.1,0.8,1.3,3.2
3,Steven Adams,C,25,OKC,80,80,33.4,6.0,10.1,.595,...,.500,4.9,4.6,9.5,1.6,1.5,1.0,1.7,2.6,13.9
4,Bam Adebayo,C,21,MIA,82,28,23.3,3.4,5.9,.576,...,.735,2.0,5.3,7.3,2.2,0.9,0.8,1.5,2.5,8.9
5,Deng Adel,SF,21,CLE,19,3,10.2,0.6,1.9,.306,...,1.0,0.2,0.8,1.0,0.3,0.1,0.2,0.3,0.7,1.7
6,DeVaughn Akoon-Purcell,SG,25,DEN,7,0,3.1,0.4,1.4,.300,...,.500,0.1,0.4,0.6,0.9,0.3,0.0,0.3,0.6,1.0
7,LaMarcus Aldridge,C,33,SAS,81,81,33.2,8.4,16.3,.519,...,.847,3.1,6.1,9.2,2.4,0.5,1.3,1.8,2.2,21.3
8,Rawle Alkins,SG,21,CHI,10,1,12.0,1.3,3.9,.333,...,.667,1.1,1.5,2.6,1.3,0.1,0.0,0.8,0.7,3.7
9,Grayson Allen,SG,23,UTA,38,2,10.9,1.8,4.7,.376,...,.750,0.1,0.5,0.6,0.7,0.2,0.2,0.9,1.2,5.6


In [6]:
#We are now going to accumulate a dataset of player statistics for the last , both regular season
#and playoff numbers.  It is important to note that basketball-reference labels the 2010-2011 without 
#hyphenation meaning that the 2010-2011 regular season is just labeled as 2011.  We will be using this labeling
#convention to assign a season to each entry in the entire dataset.

years = [x for x in range(2011,2021,1)]
regular_season_2010s = pd.DataFrame() #will contain aggregated output

In [7]:
#The code here, for the most part, is almost exactly the same as the code given above – the main difference is
#that we will be iterating through a list of years (seasons) and combining the statistics into a single 
#dataframe.

for year in years:

    # URL page we will scraping
    url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html".format(year)

    # this is the HTML from the given URL
    html = urlopen(url)
    soup = BeautifulSoup(html)

    # use findALL() to get the column headers
    soup.findAll('tr', limit=2)

    # use getText()to extract the text we need into a list
    headers = [th.getText() for th in soup.findAll('tr', limit=2)[0].findAll('th')]

    # exclude the first column as we will not need the ranking order from Basketball Reference for the analysis
    headers = headers[1:]
    
    # avoid the first header row
    rows = soup.findAll('tr')[1:]
    player_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

    stats = pd.DataFrame(player_stats, columns = headers)
    stats['Year'] = str(year)
    regular_season_2010s = pd.concat([regular_season_2010s,stats])

In [8]:
#Let's see if it works...
regular_season_2010s

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
0,Jeff Adrien,PF,24,GSW,23,0,8.5,1.0,2.3,.426,...,1.0,1.5,2.5,0.4,0.2,0.2,0.4,1.2,2.5,2011
1,Arron Afflalo,SG,25,DEN,69,69,33.7,4.5,9.1,.498,...,0.7,3.0,3.6,2.4,0.5,0.4,1.0,2.2,12.6,2011
2,Maurice Ager,SG,26,MIN,4,0,7.3,1.5,2.8,.545,...,0.0,0.5,0.5,0.3,0.3,0.0,1.0,1.0,3.8,2011
3,Alexis Ajinça,C,22,TOT,34,2,10.0,1.7,3.9,.444,...,0.5,1.8,2.3,0.3,0.3,0.6,0.5,2.1,4.2,2011
4,Alexis Ajinça,C,22,DAL,10,2,7.5,1.2,3.2,.375,...,0.5,1.2,1.7,0.2,0.3,0.5,0.1,1.3,2.9,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
672,Trae Young,PG,21,ATL,60,60,35.3,9.1,20.8,.437,...,0.5,3.7,4.3,9.3,1.1,0.1,4.8,1.7,29.6,2020
673,Cody Zeller,C,27,CHO,58,39,23.1,4.3,8.3,.524,...,2.8,4.3,7.1,1.5,0.7,0.4,1.3,2.4,11.1,2020
674,Tyler Zeller,C,30,SAS,2,0,2.0,0.5,2.0,.250,...,1.5,0.5,2.0,0.0,0.0,0.0,0.0,0.0,1.0,2020
675,Ante Žižić,C,23,CLE,22,0,10.0,1.9,3.3,.569,...,0.8,2.2,3.0,0.3,0.3,0.2,0.5,1.2,4.4,2020


In [9]:
#Great! It looks like we have our dataset with the years labeled as intended.  Now let's do the same create the
#same dataset but with playoff data only.  We will be reusing the above code – the only difference will be the
#url from which we are collecting statistics and the dataframe to which we are accumulating.

playoffs_2010s = pd.DataFrame()

for year in years:

    # URL page we will scraping
    url = "https://www.basketball-reference.com/playoffs/NBA_{}_per_game.html".format(year)

    # this is the HTML from the given URL
    html = urlopen(url)
    soup = BeautifulSoup(html)

    # use findALL() to get the column headers
    soup.findAll('tr', limit=2)

    # use getText()to extract the text we need into a list
    headers = [th.getText() for th in soup.findAll('tr', limit=2)[0].findAll('th')]

    # exclude the first column as we will not need the ranking order from Basketball Reference for the analysis
    headers = headers[1:]
    
    # avoid the first header row
    rows = soup.findAll('tr')[1:]
    player_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

    stats = pd.DataFrame(player_stats, columns = headers)
    stats['Year'] = str(year)
    playoffs_2010s = pd.concat([playoffs_2010s,stats])

In [10]:
#Let's take a look...
playoffs_2010s

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
0,Arron Afflalo,SG,25,DEN,3,3,28.3,4.0,11.3,.353,...,0.0,3.0,3.0,2.3,0.0,0.0,1.0,3.3,11.3,2011
1,LaMarcus Aldridge,PF,25,POR,6,6,43.0,8.8,19.2,.461,...,2.7,4.8,7.5,1.3,1.3,1.7,1.5,3.2,20.8,2011
2,Malik Allen,PF,32,ORL,1,0,7.0,0.0,1.0,.000,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,2011
3,Ray Allen*,SG,35,BOS,9,9,40.1,6.3,12.1,.523,...,0.6,3.2,3.8,2.4,1.2,0.1,1.1,2.3,18.9,2011
4,Tony Allen,SG,29,MEM,13,13,26.9,3.3,7.8,.426,...,1.2,1.8,2.9,1.5,1.9,0.4,1.8,3.3,8.8,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,Marvin Williams,PF,33,MIL,10,0,17.9,2.1,4.7,.447,...,1.3,3.5,4.8,0.9,0.5,0.3,0.3,1.4,5.5,2020
223,Robert Williams,C,22,BOS,13,0,11.5,1.8,2.4,.742,...,1.7,2.2,3.9,0.8,0.2,0.5,0.5,1.4,3.7,2020
224,Nigel Williams-Goss,PG,25,UTA,1,0,2.0,0.0,0.0,,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2020
225,Delon Wright,PG,27,DAL,4,0,13.3,1.5,2.5,.600,...,0.3,0.5,0.8,1.8,1.3,0.0,1.3,0.5,4.0,2020


In [11]:
#There are some entries where the Player name is None (a null value) – I believe this is due to intermediate 
#headers on basketball-reference.  We will remove those from both datasets

regular_season_2010s = regular_season_2010s[regular_season_2010s.Player.notnull()]
playoffs_2010s = playoffs_2010s[playoffs_2010s.Player.notnull()]

In [12]:
#Awesome.  Let's write these to a file so that we can export the datasets

regular_season_2010s.to_csv(r'/Users/matthewzimolzak/Zhou_Zimolzak_project/regular_season_2010s.csv', 
                            index = False)

playoffs_2010s.to_csv(r'/Users/matthewzimolzak/Zhou_Zimolzak_project/playoffs_2010s.csv', index = False)

# Accessing sportsdata.io

In [13]:
# Get Players data from sportsdata.io via API
# And check what we have in the columns
res = requests.get('https://api.sportsdata.io/v3/nba/scores/json/Players?key=a3824595f2f740dbb21dd847e49ba332')
data = json.loads(res.text)
players = pd.json_normalize(data)
players.columns

Index(['PlayerID', 'SportsDataID', 'Status', 'TeamID', 'Team', 'Jersey',
       'PositionCategory', 'Position', 'FirstName', 'LastName', 'Height',
       'Weight', 'BirthDate', 'BirthCity', 'BirthState', 'BirthCountry',
       'HighSchool', 'College', 'Salary', 'PhotoUrl', 'Experience',
       'SportRadarPlayerID', 'RotoworldPlayerID', 'RotoWirePlayerID',
       'FantasyAlarmPlayerID', 'StatsPlayerID', 'SportsDirectPlayerID',
       'XmlTeamPlayerID', 'InjuryStatus', 'InjuryBodyPart', 'InjuryStartDate',
       'InjuryNotes', 'FanDuelPlayerID', 'DraftKingsPlayerID', 'YahooPlayerID',
       'FanDuelName', 'DraftKingsName', 'YahooName', 'DepthChartPosition',
       'DepthChartOrder', 'GlobalTeamID', 'FantasyDraftName',
       'FantasyDraftPlayerID', 'UsaTodayPlayerID', 'UsaTodayHeadshotUrl',
       'UsaTodayHeadshotNoBackgroundUrl', 'UsaTodayHeadshotUpdated',
       'UsaTodayHeadshotNoBackgroundUpdated', 'NbaDotComPlayerID'],
      dtype='object')

In [14]:
# Extract those we are interested in and rename the columns into a more intuitive way
# Change the 'Name' into 'Player' for later merging convinience
players = players[['FantasyDraftName','PositionCategory', 'Position','Height','Weight','Salary','Experience']]
players = players.dropna()
players = players.sort_values('Salary', ascending = False)
players.columns = ['Player','PositionCategory', 'Position','Height','Weight','Salary','Experience']
players = players.reset_index(drop = True)
players.head(10)

,Player,PositionCategory,Position,Height,Weight,Salary,Experience
0,Stephen Curry,G,PG,75,185,43006362.0,11
1,Chris Paul,G,PG,73,175,41358814.0,15
2,Russell Westbrook,G,PG,75,200,41358814.0,12
3,John Wall,G,PG,75,210,41254920.0,9
4,James Harden,G,SG,77,220,41254920.0,11
5,Kevin Durant,F,SF,82,240,40108950.0,12
6,LeBron James,F,SF,81,250,39219565.0,17
7,Blake Griffin,F,PF,81,250,36810996.0,10
8,Paul George,F,SF,80,220,35450412.0,10
9,Klay Thompson,G,SG,78,215,35361360.0,8


In [15]:
# Get team data from sportsdata.io
# Extract those columns we are interested in
res = requests.get('https://api.sportsdata.io/v3/nba/scores/json/teams?key=a3824595f2f740dbb21dd847e49ba332')
data = json.loads(res.text)
teams = pd.json_normalize(data)
teams = teams[['TeamID', 'Key','City', 'Name','Conference', 'Division', 'PrimaryColor', 'SecondaryColor',
       'TertiaryColor', 'QuaternaryColor']]
teams = teams.rename(columns={'Key':'Tm'})
teams.head()

,TeamID,Tm,City,Name,Conference,Division,PrimaryColor,SecondaryColor,TertiaryColor,QuaternaryColor
0,1,WAS,Washington,Wizards,Eastern,Southeast,002B5C,E31837,C4CED4,FFFFFF
1,2,CHA,Charlotte,Hornets,Eastern,Southeast,1D1160,00788C,A1A1A4,7AB2DD
2,3,ATL,Atlanta,Hawks,Eastern,Southeast,C8102E,FDB927,000000,9EA2A2
3,4,MIA,Miami,Heat,Eastern,Southeast,000000,98002E,F9A01B,None
4,5,ORL,Orlando,Magic,Eastern,Southeast,0077C0,000000,C4CED4,


In [16]:
# Get stadium data from sportsdata.io
res = requests.get('https://api.sportsdata.io/v3/nba/scores/json/Stadiums?key=a3824595f2f740dbb21dd847e49ba332')
data = json.loads(res.text)
stadiums = pd.json_normalize(data)
stadiums.head()

,StadiumID,Active,Name,Address,City,State,Zip,Country,Capacity,GeoLat,GeoLong
0,1,True,Capital One Arena,601 F St. N.W.,Washington,DC,20004,USA,20290.0,38.898056,-77.020833
1,2,True,Spectrum Center,330 E. Trade St.,Charlotte,NC,28202,USA,19026.0,35.225000,-80.839167
2,3,True,State Farm Arena,1 State Farm Drive,Atlanta,GA,30303,USA,18118.0,33.757222,-84.396389
3,4,True,American Airlines Arena,601 Biscayne Blvd.,Miami,FL,33132,USA,19600.0,25.781389,-80.188056
4,5,True,Amway Center,400 W. Church St.,Orlando,FL,32801,USA,18846.0,28.539167,-81.383611


In [17]:
#Write to CSV files

players.to_csv(r'/Users/matthewzimolzak/Zhou_Zimolzak_project/player_data.csv', 
                            index = False)

teams.to_csv(r'/Users/matthewzimolzak/Zhou_Zimolzak_project/team_data.csv', index = False)

stadiums.to_csv(r'/Users/matthewzimolzak/Zhou_Zimolzak_project/stadium_data.csv', index = False)